https://www.kaggle.com/arashnic/data-on-covid19-variants-in-the-eueea-data  
Country: [String]  
country_code: 2-letter ISO country code [String]  
year_week: yyyy-Www  
Source: Data source, either GISAID EpiCoV database or TESSy. [String]  
new_cases: Weekly number of new confirmed cases. Set to zero in the event that countries have negative case counts due to retrospective correction of data. [Numeric]  
number_sequenced: Weekly number of sequences carried out [Numeric]  
percentcasessequenced 100 x newcases/numbersequenced. [Numeric]  
validdenominator: GISAID data: TRUE , TESSY data: FALSE if there are discrepancies in the data reported for a given week, such as where the sum of numberdetectionsvariant across all variants exceeds numbersequenced (aggregate data), or where no sequences have been reported that are coded as
‘wild type’ (case -based data). [Numeric]  
Variant: Each VOC, Other or UNK [Numeric]  
numberdetectionsvariant: Number of detections reported of the variant [Numeric]  
percentvariant : 100 x numberdetectionsvariant/ numbersequenced. Np value given if valid_denominator == FALSE [Numeric]

In [1]:
import pandas as pd
#import csv and sort by variant+week, remove TESSy data
variant_df=pd.read_csv("variants.csv",header=0);
variant_df=variant_df.drop(columns=["country_code","valid_denominator"]);
variant_df=variant_df.sort_values(by=['year_week','variant']);
variant_df=variant_df.loc[variant_df["source"]=="GISAID"];